TF

In [1]:
import seismic_ml_kernel

In [2]:
#train (train_data_dir, validation_data_dir, image_size)
#test (validation_data_dir + 'noise/', image_size)

In [3]:
from keras.datasets import mnist # subroutines for fetching the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data() # fetch MNIST data

#mn (X_train, y_train, X_test, y_test)

Using TensorFlow backend.


In [4]:
from keras.datasets import cifar10 # subroutines for fetching the CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data() # fetch CIFAR-10 data

#ci (X_train, y_train, X_test, y_test)

In [ ]:
def seis_dataset ():
    import numpy as np
    import seismic_handler
    
    noisers = [None, seismic_handler.FreqNoiser()]
    
    images_dir = 'images/'

    import os
    if os.path.exists(images_dir):
        import shutil
        shutil.rmtree(images_dir)

    if not os.path.exists(images_dir):
        os.makedirs(images_dir)

    train_data_dir = images_dir + 'train/'
    validation_data_dir = images_dir + 'validation/'

    if not os.path.exists(train_data_dir):
        os.makedirs(train_data_dir)

    if not os.path.exists(validation_data_dir):
        os.makedirs(validation_data_dir)
    
    handler = seismic_handler.SeismicPrestack('data/prestack.sgy', None)
    handlerNoise = seismic_handler.SeismicPrestack('data/prestack.sgy', seismic_handler.FreqNoiser())
    
    from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

    X_train = []
    y_train = []
    X_test = [] 
    y_test = []
    
    counter = 0
    
    vals = handler.getHeaderVals ('original_field_record_number')
    valsNoise = handlerNoise.getHeaderVals ('original_field_record_number')
            
    for v in vals:

        parts = handler.readGatherParts (v, 50, 0.5)
        partsNoise = handlerNoise.readGatherParts (v, 50, 0.5)
        if len (partsNoise) != len (parts):
            exit ()
        
        nparts = len(parts)

        sh = parts[0].shape
        for i in range(nparts):                
            if i % 2 == 0:
                data = partsNoise[i]
                category = 1
            else:
                category = 0
                data = parts[i]
                
            if data.shape != sh:
                throw ('wrong shapes')

            image = seismic_handler.convert_to_image (data.T)
            data = img_to_array (image)

            if i < nparts*0.8:
                X_train.append(data)
                y_train.append (category)
                png_name = train_data_dir
            else:
                X_test.append(data)
                y_test.append(category)   
                png_name = validation_data_dir
                
            if category == 1:
                png_name += 'noise/'
            else:
                png_name += 'norm/'    
            
            if not os.path.exists(png_name):
                os.makedirs(png_name)
                
            png_name += str(counter) + '.png'
            image.save (png_name)
                
            counter = counter+1
    
    y_test = np.array(y_test)
    y_train = np.array(y_train)
    X_train = np.array(X_train)
    X_test = np.array(X_test)
    
    
    if len(X_train) != len(y_train):
        print ('len(X_train) != len(y_train)')
        return
    if len(X_test) != len(y_test):
        print ('len(X_test) != len(y_test)')
        return

    return (X_train, y_train), (X_test, y_test)

In [ ]:
import seismic_ml_kernel

(X_train, y_train), (X_test, y_test) = seis_dataset ()

seismic_ml_kernel.ci (X_train, y_train, X_test, y_test)